<a href="https://colab.research.google.com/github/io-uty/2024-spark/blob/main/io-uty/MovieDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from collections import Counter

In [37]:
ls

movie.csv         ratings.csv   spark-3.5.1-bin-hadoop3/     user.csv
movie_genres.csv  sample_data/  spark-3.5.1-bin-hadoop3.tgz


In [38]:
pwd

'/content'

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [8]:
!ls

sample_data  spark-3.5.1-bin-hadoop3  spark-3.5.1-bin-hadoop3.tgz


In [40]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [41]:
# Downloading and preprocessing Cars Data downloaded origianlly from https://perso.telecom-paristech.fr/eagan/class/igr204/datasets
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv

--2024-07-17 05:35:39--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv [following]
--2024-07-17 05:35:40--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22608 (22K) [text/csv]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  22.08K  --.-KB/s    in 0.005s  

2024-07-17 05:35:40 (3.98 MB/s) - ‘cars.csv’ saved [22608/22608]



In [42]:
!ls

cars.csv   movie_genres.csv  sample_data	      spark-3.5.1-bin-hadoop3.tgz
movie.csv  ratings.csv	     spark-3.5.1-bin-hadoop3  user.csv


In [43]:
!wget movie.csv

--2024-07-17 05:35:43--  http://movie.csv/
Resolving movie.csv (movie.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘movie.csv’


In [44]:
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/movie.csv

--2024-07-17 05:35:45--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/movie.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/movie.csv [following]
--2024-07-17 05:35:45--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/movie.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-07-17 05:35:45 ERROR 404: Not Found.



In [45]:
!wget /content/movie.csv

/content/movie.csv: Scheme missing.


In [46]:
!wget https://content/movie.csv

--2024-07-17 05:35:51--  https://content/movie.csv
Resolving content (content)... failed: Name or service not known.
wget: unable to resolve host address ‘content’


In [47]:
df = spark.read.csv('movie.csv', header=True, sep=";")
df.show(5)

+-------------------------------------------------------------+
|movie id,movie title,release date,video release date,IMDb URL|
+-------------------------------------------------------------+
|                                         1,Toy Story (1995...|
|                                         2,GoldenEye (1995...|
|                                         3,Four Rooms (199...|
|                                         4,Get Shorty (199...|
|                                         5,Copycat (1995),...|
+-------------------------------------------------------------+
only showing top 5 rows



In [48]:
!ls


cars.csv   movie_genres.csv  sample_data	      spark-3.5.1-bin-hadoop3.tgz
movie.csv  ratings.csv	     spark-3.5.1-bin-hadoop3  user.csv


In [49]:
df = spark.read.csv('movie.csv', header=True, sep=",")
df.show(5)

+--------+-----------------+------------+------------------+--------------------+
|movie id|      movie title|release date|video release date|            IMDb URL|
+--------+-----------------+------------+------------------+--------------------+
|       1| Toy Story (1995)|   01-Jan-95|         01-Jan-95|http://us.imdb.co...|
|       2| GoldenEye (1995)|   01-Jan-95|         01-Jan-95|http://us.imdb.co...|
|       3|Four Rooms (1995)|   01-Jan-95|         01-Jan-95|http://us.imdb.co...|
|       4|Get Shorty (1995)|   01-Jan-95|         01-Jan-95|http://us.imdb.co...|
|       5|   Copycat (1995)|   01-Jan-95|         01-Jan-95|http://us.imdb.co...|
+--------+-----------------+------------+------------------+--------------------+
only showing top 5 rows



In [50]:
dfUser = spark.read.csv('user.csv', header = True, sep = ",")
dfRatings = spark.read.csv('ratings.csv', header = True, sep = ",")
dfMovie = spark.read.csv('movie.csv', header = True, sep = ",")
dfMovieGenres = spark.read.csv('movie_genres.csv', header = True, sep = ",")
dfMovieGenres.show(5)
dfUser.show(5)
dfRatings.show(5)
dfMovie.show(5)

+--------+-------+-------+
|mgenreId|movieId|  genre|
+--------+-------+-------+
|       1|    267|unknown|
|       2|   1373|unknown|
|       3|      2| Action|
|       4|      4| Action|
|       5|     17| Action|
+--------+-------+-------+
only showing top 5 rows

+-------+---+------+----------+--------+
|user id|age|gender|occupation|zip code|
+-------+---+------+----------+--------+
|      1| 24|     M|technician|   85711|
|      2| 53|     F|     other|   94043|
|      3| 23|     M|    writer|   32067|
|      4| 24|     M|technician|   43537|
|      5| 33|     F|     other|   15213|
+-------+---+------+----------+--------+
only showing top 5 rows

+--------+------+-------+------+----------------+
|ratingId|userId|movieId|rating|       timestamp|
+--------+------+-------+------+----------------+
|       1|   196|    242|     3|1997-12-04 15:55|
|       2|   186|    302|     3|1998-04-04 19:22|
|       3|    22|    377|     1| 1997-11-07 7:18|
|       4|   244|     51|     2| 1997-

In [51]:
dfMovieGenres.dropDuplicates(subset=['movieId']).show()

+--------+-------+----------+
|mgenreId|movieId|     genre|
+--------+-------+----------+
|     389|      1| Animation|
|    1223|     10|     Drama|
|    1070|    100|     Crime|
|     855|   1000|    Comedy|
|     856|   1001|    Comedy|
|     857|   1002|    Comedy|
|     519|   1003|Children's|
|    1602|   1004|     Drama|
|    1603|   1005|     Drama|
|    1604|   1006|     Drama|
|     859|   1007|    Comedy|
|    1605|   1008|     Drama|
|    1606|   1009|     Drama|
|      26|    101|    Action|
|    1607|   1010|     Drama|
|    1122|   1011|     Crime|
|     860|   1012|    Comedy|
|     191|   1013|    Action|
|     861|   1014|    Comedy|
|     520|   1015|Children's|
+--------+-------+----------+
only showing top 20 rows



In [2]:
dfMovieGenres = spark.read.csv('movie_genres.csv', header = True, sep = ",")


NameError: name 'spark' is not defined

In [53]:
dfMovieGenres = dfMovieGenres.dropDuplicates(['movieId', 'genre'])

In [54]:
from pyspark.sql.functions import col
FemaleUsers=dfUser.filter(col('gender') == 'F')
FemaleUsers.show(5)

+-------+---+------+----------+--------+
|user id|age|gender|occupation|zip code|
+-------+---+------+----------+--------+
|      2| 53|     F|     other|   94043|
|      5| 33|     F|     other|   15213|
|     11| 39|     F|     other|   30329|
|     12| 28|     F|     other|    6405|
|     15| 49|     F|  educator|   97301|
+-------+---+------+----------+--------+
only showing top 5 rows



In [55]:
FemaleUsers.drop('occupation')\
.drop('zip code')

user id,age,gender
2,53,F
5,33,F
11,39,F
12,28,F
15,49,F
18,35,F
20,42,F
23,30,F
24,21,F
27,40,F


In [56]:
FilteredFemaleUsers = FemaleUsers.filter(col('age')>=20)
FilteredFemaleUsers = FemaleUsers.filter(col('age')<=30)
FilteredFemaleUsers = FilteredFemaleUsers.drop('occupation')
FilteredFemaleUsers = FilteredFemaleUsers.drop('zip code')
FilteredFemaleUsers.show(5)

+-------+---+------+
|user id|age|gender|
+-------+---+------+
|     12| 28|     F|
|     23| 30|     F|
|     24| 21|     F|
|     32| 28|     F|
|     35| 20|     F|
+-------+---+------+
only showing top 5 rows



In [83]:
femaleUsers = FilteredFemaleUsers
femaleUsers = femaleUsers.withColumnRenamed('user id', 'userId')
femaleUsers.show(5)

+------+---+------+
|userId|age|gender|
+------+---+------+
|    12| 28|     F|
|    23| 30|     F|
|    24| 21|     F|
|    32| 28|     F|
|    35| 20|     F|
+------+---+------+
only showing top 5 rows



In [84]:
userRating=femaleUsers.join(dfRatings, femaleUsers.userId == dfRatings.userId, 'inner').select(femaleUsers['userId'],dfRatings['movieId'],dfRatings['rating'])

In [85]:
userRating.show(5)
dfMovieGenres.show(5)
count = dfMovieGenres.groupBy('genre').count()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   253|    465|     5|
|    62|    257|     2|
|    38|     95|     5|
|   223|    274|     4|
|   241|    690|     2|
+------+-------+------+
only showing top 5 rows

+--------+-------+----------+
|mgenreId|movieId|     genre|
+--------+-------+----------+
|     389|      1| Animation|
|     431|      1|Children's|
|     553|      1|    Comedy|
|    1223|     10|     Drama|
|    2796|     10|       War|
+--------+-------+----------+
only showing top 5 rows



In [100]:
movierate=dfMovieGenres.join(userRating, dfMovieGenres.movieId == userRating.movieId, 'inner').select(dfMovieGenres['movieId'],dfMovieGenres['genre'],userRating['rating'])

In [180]:
movierate.show()
genrecount = movierate.groupBy('genre','rating').count()
genrecount.show()

+-------+----------+------+
|movieId|     genre|rating|
+-------+----------+------+
|    465|Children's|     5|
|    465| Adventure|     5|
|    465|   Romance|     5|
|    257|    Comedy|     2|
|    257| Adventure|     2|
|    257|    Action|     2|
|    257|    Sci-Fi|     2|
|     95| Animation|     5|
|     95|    Comedy|     5|
|     95|Children's|     5|
|     95|   Musical|     5|
|    274|   Romance|     4|
|    274|    Comedy|     4|
|    690|       War|     2|
|    690|     Drama|     2|
|    498|    Action|     4|
|    498|   Romance|     4|
|    498|       War|     4|
|    498| Adventure|     4|
|    382|     Drama|     3|
+-------+----------+------+
only showing top 20 rows

+-----------+------+-----+
|      genre|rating|count|
+-----------+------+-----+
|  Animation|     1|   46|
|   Thriller|     2|  341|
|     Horror|     1|  122|
|      Drama|     3| 1295|
|     Sci-Fi|     3|  401|
| Children's|     1|  126|
|Documentary|     2|    7|
|    Fantasy|     3|   72|
|    

In [181]:
genrecount = genrecount.orderBy('genre', 'rating')
genrecount.show()

+----------+------+-----+
|     genre|rating|count|
+----------+------+-----+
|    Action|     1|  265|
|    Action|     2|  361|
|    Action|     3|  845|
|    Action|     4|  911|
|    Action|     5|  651|
| Adventure|     1|  161|
| Adventure|     2|  192|
| Adventure|     3|  505|
| Adventure|     4|  542|
| Adventure|     5|  398|
| Animation|     1|   46|
| Animation|     2|   50|
| Animation|     3|  187|
| Animation|     4|  221|
| Animation|     5|  153|
|Children's|     1|  126|
|Children's|     2|  158|
|Children's|     3|  419|
|Children's|     4|  438|
|Children's|     5|  264|
+----------+------+-----+
only showing top 20 rows



In [205]:
genrecount = genrecount.withColumn("rating_count_product", col("rating") * col("count"))
genrecount.show()

+----------+------+-----+--------------------+
|     genre|rating|count|rating_count_product|
+----------+------+-----+--------------------+
|    Action|     1|  265|               265.0|
|    Action|     2|  361|               722.0|
|    Action|     3|  845|              2535.0|
|    Action|     4|  911|              3644.0|
|    Action|     5|  651|              3255.0|
| Adventure|     1|  161|               161.0|
| Adventure|     2|  192|               384.0|
| Adventure|     3|  505|              1515.0|
| Adventure|     4|  542|              2168.0|
| Adventure|     5|  398|              1990.0|
| Animation|     1|   46|                46.0|
| Animation|     2|   50|               100.0|
| Animation|     3|  187|               561.0|
| Animation|     4|  221|               884.0|
| Animation|     5|  153|               765.0|
|Children's|     1|  126|               126.0|
|Children's|     2|  158|               316.0|
|Children's|     3|  419|              1257.0|
|Children's| 

In [226]:
genre_total_count = genrecount.groupBy("genre").agg(_sum("count").alias("movieCount"))

# 원래 데이터프레임과 장르별 총 count를 병합
genrecount_with_total = genrecount.join(genre_total_count, on="genre")
genrecount_with_total.show()

+-----------+------+-----+--------------------+----------+
|      genre|rating|count|rating_count_product|movieCount|
+-----------+------+-----+--------------------+----------+
|  Animation|     1|   46|                46.0|       657|
|   Thriller|     2|  341|               682.0|      2766|
|     Horror|     1|  122|               122.0|       755|
|      Drama|     3| 1295|              3885.0|      5319|
|     Sci-Fi|     3|  401|              1203.0|      1482|
| Children's|     1|  126|               126.0|      1405|
|Documentary|     2|    7|                14.0|        77|
|    Fantasy|     3|   72|               216.0|       226|
|     Comedy|     2|  521|              1042.0|      4345|
|    Musical|     5|  198|               990.0|       825|
| Children's|     2|  158|               316.0|      1405|
|    Mystery|     5|  115|               575.0|       601|
|    Romance|     5|  797|              3985.0|      2990|
|    Romance|     1|  215|               215.0|      299

In [227]:
genrecount_with_total = genrecount_with_total.withColumn(
    "normalized_rating", col("rating") * col("count") / col("movieCount")
)

In [228]:
genre_total_rating = genrecount_with_total.groupBy("genre").agg(_sum("normalized_rating").alias("averageRating")) # Use genrecount_with_total instead of genrecount
genrecount_with_total = genrecount.join(genre_total_rating, on="genre")
genrecount_with_total.show()

+---------+------+-----+--------------------+------------------+
|    genre|rating|count|rating_count_product|     averageRating|
+---------+------+-----+--------------------+------------------+
|    Crime|     5|  196|               980.0|3.4906490649064903|
|    Crime|     4|  308|              1232.0|3.4906490649064903|
|    Crime|     3|  225|               675.0|3.4906490649064903|
|    Crime|     2|  106|               212.0|3.4906490649064903|
|    Crime|     1|   74|                74.0|3.4906490649064903|
|  Romance|     4|  983|              3932.0|3.6227424749163877|
|  Romance|     2|  285|               570.0|3.6227424749163877|
|  Romance|     3|  710|              2130.0|3.6227424749163877|
|  Romance|     1|  215|               215.0|3.6227424749163877|
|  Romance|     5|  797|              3985.0|3.6227424749163877|
| Thriller|     3|  771|              2313.0|  3.44179320318149|
| Thriller|     5|  575|              2875.0|  3.44179320318149|
| Thriller|     4|  857| 

In [229]:
genre_total_counting = genrecount_with_total.groupBy("genre").agg(_sum("count").alias("movieCount")) # Use genrecount_with_total instead of genrecount


In [230]:
genrecount_with_total.show()

+---------+------+-----+--------------------+------------------+
|    genre|rating|count|rating_count_product|     averageRating|
+---------+------+-----+--------------------+------------------+
|    Crime|     5|  196|               980.0|3.4906490649064903|
|    Crime|     4|  308|              1232.0|3.4906490649064903|
|    Crime|     3|  225|               675.0|3.4906490649064903|
|    Crime|     2|  106|               212.0|3.4906490649064903|
|    Crime|     1|   74|                74.0|3.4906490649064903|
|  Romance|     4|  983|              3932.0|3.6227424749163877|
|  Romance|     2|  285|               570.0|3.6227424749163877|
|  Romance|     3|  710|              2130.0|3.6227424749163877|
|  Romance|     1|  215|               215.0|3.6227424749163877|
|  Romance|     5|  797|              3985.0|3.6227424749163877|
| Thriller|     3|  771|              2313.0|  3.44179320318149|
| Thriller|     5|  575|              2875.0|  3.44179320318149|
| Thriller|     4|  857| 

In [232]:
genrecount_with_total = genrecount_with_total.join(genre_total_counting, on="genre")

In [233]:
genrecont_with_total = genrecount_with_total.orderBy("averageRating", ascending=True)
genrecount_with_total = genrecount_with_total.drop("rating")\
.drop("rating_count_product")
genrecount_with_total.show()

+----------+-----+------------------+----------+
|     genre|count|     averageRating|movieCount|
+----------+-----+------------------+----------+
|    Action|  911|3.4358720738542696|      3033|
|    Action|  845|3.4358720738542696|      3033|
|    Action|  265|3.4358720738542696|      3033|
|    Action|  361|3.4358720738542696|      3033|
|    Action|  651|3.4358720738542696|      3033|
| Adventure|  398|3.4582869855394884|      1798|
| Adventure|  505|3.4582869855394884|      1798|
| Adventure|  192|3.4582869855394884|      1798|
| Adventure|  161|3.4582869855394884|      1798|
| Adventure|  542|3.4582869855394884|      1798|
| Animation|  187|3.5859969558599696|       657|
| Animation|   50|3.5859969558599696|       657|
| Animation|  153|3.5859969558599696|       657|
| Animation|  221|3.5859969558599696|       657|
| Animation|   46|3.5859969558599696|       657|
|Children's|  419| 3.395729537366548|      1405|
|Children's|  438| 3.395729537366548|      1405|
|Children's|  264| 3

In [194]:
genrecount_with_total.show()

+---------+------+-----+--------------------+----------+
|    genre|rating|count|rating_count_product|movieCount|
+---------+------+-----+--------------------+----------+
|    Crime|     5|  196|               980.0|       909|
|    Crime|     4|  308|              1232.0|       909|
|    Crime|     3|  225|               675.0|       909|
|    Crime|     2|  106|               212.0|       909|
|    Crime|     1|   74|                74.0|       909|
|  Romance|     4|  983|              3932.0|      2990|
|  Romance|     2|  285|               570.0|      2990|
|  Romance|     3|  710|              2130.0|      2990|
|  Romance|     1|  215|               215.0|      2990|
|  Romance|     5|  797|              3985.0|      2990|
| Thriller|     3|  771|              2313.0|      2766|
| Thriller|     5|  575|              2875.0|      2766|
| Thriller|     4|  857|              3428.0|      2766|
| Thriller|     1|  222|               222.0|      2766|
| Thriller|     2|  341|       

In [1]:
genrecount_with_total = genrecount_with_total.drop("rating")\
.drop("rating_count_product")

NameError: name 'genrecount_with_total' is not defined